# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [29]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [30]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [31]:
spaceship = spaceship.dropna()
spaceship['Cabin']= spaceship['Cabin'].str[0]
spaceship= spaceship.drop(['PassengerId','Name'], axis=1)

In [32]:
spaceship_cat= spaceship.select_dtypes(include=['object'])
spaceship_cat= pd.get_dummies(spaceship_cat, dtype = int)
spaceship_num = spaceship.select_dtypes(include=['number'])

In [33]:
spaceship_combined = pd.concat([spaceship_num, spaceship_cat], axis=1)
spaceship_combined

Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  \
0     39.0          0.0        0.0           0.0     0.0     0.0   
1     24.0        109.0        9.0          25.0   549.0    44.0   
2     58.0         43.0     3576.0           0.0  6715.0    49.0   
3     33.0          0.0     1283.0         371.0  3329.0   193.0   
4     16.0        303.0       70.0         151.0   565.0     2.0   
...    ...          ...        ...           ...     ...     ...   
8688  41.0          0.0     6819.0           0.0  1643.0    74.0   
8689  18.0          0.0        0.0           0.0     0.0     0.0   
8690  26.0          0.0        0.0        1872.0     1.0     0.0   
8691  32.0          0.0     1049.0           0.0   353.0  3235.0   
8692  44.0        126.0     4688.0           0.0     0.0    12.0   

      HomePlanet_Earth  HomePlanet_Europa  HomePlanet_Mars  CryoSleep_False  \
0                    0                  1                0                1   
1                    1                  0                0                1   
2                    0                  1                0                1   
3                    0                  1                0                1   
4                    1                  0                0                1   
...                ...                ...              ...              ...   
8688                 0                  1                0                1   
8689                 1                  0                0                0   
8690                 1                  0                0                1   
8691                 0                  1                0                1   
8692                 0                  1                0                1   

      ...  Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  \
0     ...        0        0        0        0        0   
1     ...        0        0        1        0        0   
2     ...        0        0        0        0        0   
3     ...        0        0        0        0        0   
4     ...        0        0        1        0        0   
...   ...      ...      ...      ...      ...      ...   
8688  ...        0        0        0        0        0   
8689  ...        0        0        0        1        0   
8690  ...        0        0        0        1        0   
8691  ...        0        1        0        0        0   
8692  ...        0        1        0        0        0   

      Destination_55 Cancri e  Destination_PSO J318.5-22  \
0                           0                          0   
1                           0                          0   
2                           0                          0   
3                           0                          0   
4                           0                          0   
...                       ...                        ...   
8688                        1                          0   
8689                        0                          1   
8690                        0                          0   
8691                        1                          0   
8692                        0                          0   

      Destination_TRAPPIST-1e  VIP_False  VIP_True  
0                           1          1         0  
1                           1          1         0  
2                           1          0         1  
3                           1          1         0  
4                           1          1         0  
...                       ...        ...       ...  
8688                        0          0         1  
8689                        0          1         0  
8690                        1          1         0  
8691                        0          1         0  
8692                        1          1         0  

[6606 rows x 24 columns]

In [34]:
# Train Test Split
features = spaceship_combined
target = spaceship["Transported"]
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

In [35]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
normalizer = MinMaxScaler()

In [36]:
normalizer.fit(X_train) 

MinMaxScaler()

In [37]:
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [38]:
gb_cl = GradientBoostingClassifier(max_depth=7,
                                   n_estimators=8)

In [39]:
gb_cl.fit(X_train_norm, y_train)

GradientBoostingClassifier(max_depth=7, n_estimators=8)

In [40]:
y_pred_gbc = gb_cl.predict(X_test_norm)

c:\Users\Pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


- Evaluate your model

In [41]:
accuracy_gbc= accuracy_score(y_test, y_pred_gbc)
accuracy_gbc

0.7866868381240545

In [42]:
report1 = classification_report(y_test, y_pred_gbc)
print("Classification Report:\n", report1)

Classification Report:
               precision    recall  f1-score   support

       False       0.80      0.76      0.78       661
        True       0.77      0.82      0.79       661

    accuracy                           0.79      1322
   macro avg       0.79      0.79      0.79      1322
weighted avg       0.79      0.79      0.79      1322



**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [43]:
#your code here

grid = {"n_estimators": [5, 10, 20,100],
        "max_leaf_nodes": [50, 100, 500, None],
        "max_depth":[6,10,50]}

- Run Grid Search

In [44]:
gb_cl = GradientBoostingClassifier()

model = GridSearchCV(estimator=gb_cl, param_grid=grid, cv=5)

model.fit(X_train_norm, y_train)

best_params = model.best_params_

best_model = model.best_estimator_

best_params, best_model

pred = best_model.predict(X_test_norm)

accuracy_gbc_best= accuracy_score(y_test, pred)

accuracy_gbc_best

c:\Users\Pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


0.7889561270801816

- Evaluate your model

In [45]:
report = classification_report(y_test, pred)

print("Classification Report:\n", report)
# Training final model

gb_cl = GradientBoostingClassifier(max_depth=6,
                                   max_leaf_nodes= 500,
                                   n_estimators=10)

gb_cl.fit(X_train_norm, y_train)

y_pred_gbc_best = gb_cl.predict(X_test_norm)

accuracy_gbc_best= accuracy_score(y_test, y_pred_gbc_best)

accuracy_gbc_best

Classification Report:
               precision    recall  f1-score   support

       False       0.80      0.77      0.79       661
        True       0.78      0.81      0.79       661

    accuracy                           0.79      1322
   macro avg       0.79      0.79      0.79      1322
weighted avg       0.79      0.79      0.79      1322



c:\Users\Pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


0.7889561270801816